In [1]:
# Import libraries
import os
import sys
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set Pandas row limit
pd.set_option('display.max_rows', 500)

# Load the data from the CSV file
price_data = pd.read_csv('./data/CCL_historical_data.csv')

In [2]:
# Add a "Change in price" column
price_data['change_in_price'] = price_data['c'].diff()
price_data.head()

,datetime,o,h,l,c,v,readable_time,date,change_in_price
0,1571299800000,41.87,41.93,41.87,41.93,400,2019-10-17 03:10:00,2019-10-17,NaN
1,1571300100000,41.94,41.94,41.88,41.88,900,2019-10-17 03:15:00,2019-10-17,-0.05
2,1571301300000,41.84,41.88,41.84,41.88,800,2019-10-17 03:35:00,2019-10-17,0.00
3,1571301600000,41.88,41.88,41.88,41.88,100,2019-10-17 03:40:00,2019-10-17,0.00
4,1571305500000,42.26,42.26,42.26,42.26,300,2019-10-17 04:45:00,2019-10-17,0.38


In [3]:
# Calculate 20 EMA (short term)
ema_20 = price_data['c'].transform(lambda x: x.ewm(span = 20).mean())
price_data['EMA_20'] = ema_20

# Calculate 200 EMA (long term)
ema_200 = price_data['c'].transform(lambda x: x.ewm(span = 200).mean())
price_data['EMA_200'] = ema_200

#  Calculate MACD 
ema_26 = price_data['c'].transform(lambda x: x.ewm(span = 26).mean())
ema_12 = price_data['c'].transform(lambda x: x.ewm(span = 12).mean())
macd = ema_12 - ema_26

ema_9_macd = macd.ewm(span = 9).mean()

# Calculate the VWAP
def vol_weighted_avg_price(date_group):
    vwap_col = []
    volume = date_group['v']
    high = date_group['h']
    low = date_group['l']
    
    # Calculate VWAP
    vwap_col = (volume * ((high + low)/2)).cumsum() / volume.cumsum()
    return pd.Series(vwap_col, index = date_group.index)
    
vwap = price_data.groupby('date').apply(vol_weighted_avg_price)
price_data['VWAP_50'] = vwap.reset_index(level=0, drop=True)

# Store the data in the data fram
price_data['MACD'] = macd
price_data['MACD_EMA'] = ema_9_macd
price_data['MACD_Diff'] = macd - ema_9_macd

price_data.tail(100)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff
25455,1593704400000,16.2300,16.3700,16.2300,16.3400,324065,2020-07-02 10:40:00,2020-07-02,0.1099,16.393022,16.662613,16.630653,-0.150032,-0.172664,0.022632
25456,1593704700000,16.3400,16.4400,16.3300,16.4250,370673,2020-07-02 10:45:00,2020-07-02,0.0850,16.396068,16.660249,16.624014,-0.129488,-0.164029,0.034541
25457,1593705000000,16.4261,16.4383,16.3200,16.3918,268503,2020-07-02 10:50:00,2020-07-02,-0.0332,16.395661,16.657578,16.619313,-0.114566,-0.154136,0.039571
25458,1593705300000,16.4000,16.4300,16.3200,16.3430,264727,2020-07-02 10:55:00,2020-07-02,-0.0488,16.390646,16.654448,16.614774,-0.105461,-0.144401,0.038940
25459,1593705600000,16.3500,16.3600,16.2500,16.3000,225856,2020-07-02 11:00:00,2020-07-02,-0.0430,16.382013,16.650921,16.609940,-0.100556,-0.135632,0.035076
25460,1593705900000,16.2934,16.3700,16.2900,16.3200,187295,2020-07-02 11:05:00,2020-07-02,0.0200,16.376107,16.647628,16.606364,-0.093972,-0.127300,0.033328
25461,1593706200000,16.3100,16.3350,16.2500,16.2800,135201,2020-07-02 11:10:00,2020-07-02,-0.0400,16.366954,16.643970,16.603496,-0.090934,-0.120027,0.029093
25462,1593706500000,16.2700,16.2700,16.1100,16.1580,427210,2020-07-02 11:15:00,2020-07-02,-0.1220,16.347054,16.639135,16.591893,-0.097249,-0.115471,0.018222
25463,1593706800000,16.1500,16.1700,16.0900,16.1500,359418,2020-07-02 11:20:00,2020-07-02,-0.0080,16.328287,16.634268,16.581241,-0.101727,-0.112723,0.010996
25464,1593707100000,16.1500,16.2400,16.1499,16.2400,234739,2020-07-02 11:25:00,2020-07-02,0.0900,16.319878,16.630345,16.575508,-0.096897,-0.109557,0.012661


In [4]:
# Indicate buy points
def determine_buy_signal(df):
    buy_signals = []
    open_price = df['o']
    close = df['c']
    macd_diff = df['MACD_Diff']
    ema_20 = df['EMA_20']
    ema_200 = df['EMA_200']
    vwap = df['VWAP_50']
    

    for op, close, macd, ema, ema_long, vwap in zip(open_price, close, macd_diff, ema_20, ema_200, vwap):
        
        # Did the candle stick open above EMA
        if (op > ema and op > ema_long and close > ema and close > ema_long and macd > 0.035 and macd < 0.06):
            buy_signals.append('BUY')
        else:
            buy_signals.append('pass')
            continue
    
#         # Is the MACD gapping but not extended
#         if (macd > 0.02 and macd < 0.04):
#             pass
#         else:
#             buy_signals.append('pass')
#             continue
        
#         # MACD extended. Likely a reveral is coming
#         if (macd > 0.04):
#             buy_signals.append('pass')
#             continue
            
#         # Is the price near the 75% VWAP? Don't buy
            
#         # Is the price near the 50% VWAP? Buy
#         if (close > vwap):
#             pass
#         else:
#             buy_signals.append('pass')
#             continue
            
#         buy_signals.append('BUY')
    
    return pd.Series(buy_signals)

buy_column = determine_buy_signal(price_data)

price_data['Buy_Signals'] = buy_column

price_data.tail(50)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy_Signals
25505,1593719400000,15.8550,15.8700,15.8100,15.8650,448158,2020-07-02 14:50:00,2020-07-02,0.0050,15.906035,16.406523,16.337951,-0.050906,-0.055933,0.005027,pass
25506,1593719700000,15.8650,15.9100,15.8400,15.8800,1060381,2020-07-02 14:55:00,2020-07-02,0.0150,15.903556,16.401284,16.319709,-0.047082,-0.054163,0.007081,pass
25507,1593720000000,15.8800,15.9500,15.8800,15.9100,504752,2020-07-02 15:00:00,2020-07-02,0.0300,15.904169,16.396395,16.312257,-0.041156,-0.051561,0.010406,pass
25508,1593720300000,15.9100,15.9400,15.9000,15.9400,9192,2020-07-02 15:05:00,2020-07-02,0.0300,15.907582,16.391854,16.312126,-0.033651,-0.047979,0.014329,pass
25509,1593720600000,15.9400,16.0000,15.9400,15.9900,13472,2020-07-02 15:10:00,2020-07-02,0.0500,15.915431,16.387856,16.311958,-0.023398,-0.043063,0.019665,pass
25510,1593720900000,15.9900,16.0000,15.9800,15.9998,8193,2020-07-02 15:15:00,2020-07-02,0.0098,15.923466,16.383994,16.311862,-0.014318,-0.037314,0.022996,pass
25511,1593721200000,15.9800,16.0000,15.9700,15.9700,19477,2020-07-02 15:20:00,2020-07-02,-0.0298,15.927898,16.379875,16.311630,-0.009417,-0.031735,0.022317,pass
25512,1593721500000,15.9998,17.1300,15.9500,17.1300,11861,2020-07-02 15:25:00,2020-07-02,1.1600,16.042384,16.387339,16.311728,0.087065,-0.007975,0.095040,pass
25513,1593721800000,15.9900,16.0000,15.9900,16.0000,6390,2020-07-02 15:30:00,2020-07-02,-1.1300,16.038347,16.383485,16.311655,0.071522,0.007925,0.063597,pass
25514,1593722100000,15.9900,16.0000,15.9600,15.9900,1726,2020-07-02 15:35:00,2020-07-02,-0.0100,16.033743,16.379570,16.311634,0.057732,0.017886,0.039846,pass


In [5]:
# Incidate sell points
def determine_sell_signal(df):
    sell_signals = []
    open_price = df['o']
    close = df['c']
    macd_diff = df['MACD_Diff']
    ema_20 = df['EMA_20']
    vwap = df['VWAP_50']

    for op, close, macd, ema, vwap in zip(open_price, close, macd_diff, ema_20, vwap):
        
        # Did the candle stick close below the EMA line
        if (close < ema):
            sell_signals.append('SELL')
            continue
            
        # MACD extended
        if (macd > 0.06):
            sell_signals.append('SELL')
            continue
        
        sell_signals.append('pass')
        
    return pd.Series(sell_signals)

sell_column = determine_sell_signal(price_data)

price_data['Sell_Signals'] = sell_column

price_data.tail(50)



,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy_Signals,Sell_Signals
25505,1593719400000,15.8550,15.8700,15.8100,15.8650,448158,2020-07-02 14:50:00,2020-07-02,0.0050,15.906035,16.406523,16.337951,-0.050906,-0.055933,0.005027,pass,SELL
25506,1593719700000,15.8650,15.9100,15.8400,15.8800,1060381,2020-07-02 14:55:00,2020-07-02,0.0150,15.903556,16.401284,16.319709,-0.047082,-0.054163,0.007081,pass,SELL
25507,1593720000000,15.8800,15.9500,15.8800,15.9100,504752,2020-07-02 15:00:00,2020-07-02,0.0300,15.904169,16.396395,16.312257,-0.041156,-0.051561,0.010406,pass,pass
25508,1593720300000,15.9100,15.9400,15.9000,15.9400,9192,2020-07-02 15:05:00,2020-07-02,0.0300,15.907582,16.391854,16.312126,-0.033651,-0.047979,0.014329,pass,pass
25509,1593720600000,15.9400,16.0000,15.9400,15.9900,13472,2020-07-02 15:10:00,2020-07-02,0.0500,15.915431,16.387856,16.311958,-0.023398,-0.043063,0.019665,pass,pass
25510,1593720900000,15.9900,16.0000,15.9800,15.9998,8193,2020-07-02 15:15:00,2020-07-02,0.0098,15.923466,16.383994,16.311862,-0.014318,-0.037314,0.022996,pass,pass
25511,1593721200000,15.9800,16.0000,15.9700,15.9700,19477,2020-07-02 15:20:00,2020-07-02,-0.0298,15.927898,16.379875,16.311630,-0.009417,-0.031735,0.022317,pass,pass
25512,1593721500000,15.9998,17.1300,15.9500,17.1300,11861,2020-07-02 15:25:00,2020-07-02,1.1600,16.042384,16.387339,16.311728,0.087065,-0.007975,0.095040,pass,SELL
25513,1593721800000,15.9900,16.0000,15.9900,16.0000,6390,2020-07-02 15:30:00,2020-07-02,-1.1300,16.038347,16.383485,16.311655,0.071522,0.007925,0.063597,pass,SELL
25514,1593722100000,15.9900,16.0000,15.9600,15.9900,1726,2020-07-02 15:35:00,2020-07-02,-0.0100,16.033743,16.379570,16.311634,0.057732,0.017886,0.039846,pass,SELL


In [6]:
# Calculate enter and exit points
# This will strictly use the buy/sell signal. As soon as a buy is reached, we buy. After that, we hold until we get a sell signal

def enter_exit_points(df):
    enter_exit_points = []
    buy_signals = df['Buy_Signals']
    sell_signals = df['Sell_Signals']
    
    holding = False
    
    for bs, ss in zip(buy_signals, sell_signals):
        
        if(not holding):
            if (bs == 'BUY'):
                enter_exit_points.append('ENTER')
                holding = True
                continue
            else:
                enter_exit_points.append('wait')
                continue
        else:
            if(ss == 'SELL'):
                enter_exit_points.append('EXIT')
                holding = False
                continue
            else:
                enter_exit_points.append('hold')
                continue
                
    return pd.Series(enter_exit_points)

enter_exit_column = enter_exit_points(price_data)

price_data['Enter_Exit'] = enter_exit_column

price_data.tail(50)

,datetime,o,h,l,c,v,readable_time,date,change_in_price,EMA_20,EMA_200,VWAP_50,MACD,MACD_EMA,MACD_Diff,Buy_Signals,Sell_Signals,Enter_Exit
25505,1593719400000,15.8550,15.8700,15.8100,15.8650,448158,2020-07-02 14:50:00,2020-07-02,0.0050,15.906035,16.406523,16.337951,-0.050906,-0.055933,0.005027,pass,SELL,wait
25506,1593719700000,15.8650,15.9100,15.8400,15.8800,1060381,2020-07-02 14:55:00,2020-07-02,0.0150,15.903556,16.401284,16.319709,-0.047082,-0.054163,0.007081,pass,SELL,wait
25507,1593720000000,15.8800,15.9500,15.8800,15.9100,504752,2020-07-02 15:00:00,2020-07-02,0.0300,15.904169,16.396395,16.312257,-0.041156,-0.051561,0.010406,pass,pass,wait
25508,1593720300000,15.9100,15.9400,15.9000,15.9400,9192,2020-07-02 15:05:00,2020-07-02,0.0300,15.907582,16.391854,16.312126,-0.033651,-0.047979,0.014329,pass,pass,wait
25509,1593720600000,15.9400,16.0000,15.9400,15.9900,13472,2020-07-02 15:10:00,2020-07-02,0.0500,15.915431,16.387856,16.311958,-0.023398,-0.043063,0.019665,pass,pass,wait
25510,1593720900000,15.9900,16.0000,15.9800,15.9998,8193,2020-07-02 15:15:00,2020-07-02,0.0098,15.923466,16.383994,16.311862,-0.014318,-0.037314,0.022996,pass,pass,wait
25511,1593721200000,15.9800,16.0000,15.9700,15.9700,19477,2020-07-02 15:20:00,2020-07-02,-0.0298,15.927898,16.379875,16.311630,-0.009417,-0.031735,0.022317,pass,pass,wait
25512,1593721500000,15.9998,17.1300,15.9500,17.1300,11861,2020-07-02 15:25:00,2020-07-02,1.1600,16.042384,16.387339,16.311728,0.087065,-0.007975,0.095040,pass,SELL,wait
25513,1593721800000,15.9900,16.0000,15.9900,16.0000,6390,2020-07-02 15:30:00,2020-07-02,-1.1300,16.038347,16.383485,16.311655,0.071522,0.007925,0.063597,pass,SELL,wait
25514,1593722100000,15.9900,16.0000,15.9600,15.9900,1726,2020-07-02 15:35:00,2020-07-02,-0.0100,16.033743,16.379570,16.311634,0.057732,0.017886,0.039846,pass,SELL,wait


In [7]:
# Add an account balance total, starting with $1000
def track_account_balance(df):
    account_balance = []
    current_balance = 1500.0
    enter_exit = df['Enter_Exit']
    close = df['c']
    
    holding = False
    account_balance.append(1500.0)
    
    for ee, close in zip(enter_exit, close):
        if(ee == 'ENTER' and holding == False):
            current_balance -= (close * 30.0)
            account_balance.append(current_balance)
            holding = True
            continue
        
        if(ee == 'EXIT' and holding == True):
            current_balance += (close * 30.0)
            account_balance.append(current_balance)
            holding = False
            continue
        
        account_balance.append(current_balance)
            
    return pd.Series(account_balance)

account_balance_col = track_account_balance(price_data)

price_data['Account_Balance'] = account_balance_col
price_data.tail(400)

# Export to CSV
price_data.to_csv("ccl.csv")